In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client()
project_id = client.project

In [ ]:
dataset_id = f"{project_id}.data_to_ai_workshop"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"

dataset = client.create_dataset(dataset, exists_ok=True)

In [ ]:
uri = "gs://labs.roitraining.com/data-to-ai-workshop/airports.csv"
table_id = f"{project_id}.data_to_ai_workshop.airports"

In [ ]:
job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.CSV,
    skip_leading_rows = 1,
    autodetect = True
)

In [ ]:
load_job = client.load_table_from_uri(uri, table_id, job_config = job_config)
load_job.result()

LoadJob<project=qwiklabs-gcp-01-a82571ac193f, location=US, id=5eee4e74-9ed5-47d2-be8a-37b531117568>

In [ ]:
%%bigquery
SELECT *
FROM `data_to_ai_workshop.airports`
LIMIT 5


Query is running:   0%|          |

Downloading:   0%|          |

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11,NA,US,US-PA,Bensalem,False,None,None,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,None,None
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435,NA,US,US-KS,Leoti,False,None,None,00AA,00AA,None,None,None
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450,NA,US,US-AK,Anchor Point,False,None,None,00AK,00AK,None,None,None
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820,NA,US,US-AL,Harvest,False,None,None,00AL,00AL,None,None,None
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80,NA,US,US-AK,King Salmon,False,None,None,00AN,00AN,None,None,None


In [ ]:
%%bigquery
SELECT type,iso_country,COUNT(*) AS airport_type_cnt
FROM `data_to_ai_workshop.airports`
GROUP BY type,iso_country
HAVING iso_country = 'US';

Query is running:   0%|          |

Downloading:   0%|          |

,type,iso_country,airport_type_cnt
0,heliport,US,8137
1,small_airport,US,15225
2,seaplane_base,US,650
3,balloonport,US,29
4,closed,US,7098
5,medium_airport,US,832
6,large_airport,US,71


In [ ]:
%%bigquery
SELECT COUNT(*) AS airport_count
FROM `data_to_ai_workshop.airports`;


Query is running:   0%|          |

Downloading:   0%|          |

,airport_count
0,82893


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `data_to_ai_workshop.large_airports` AS
SELECT
  id,
  ident,
  name,
  type,
  latitude_deg,
  longitude_deg,
  iso_country,
  iso_region,
  municipality,
  iata_code,
  icao_code
FROM `data_to_ai_workshop.airports`
WHERE iso_country = 'US'
  AND type = 'large_airport'
  AND latitude_deg IS NOT NULL
  AND longitude_deg IS NOT NULL;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  type,
  COUNT(*) AS airport_count
FROM `data_to_ai_workshop.large_airports`
GROUP BY type;


Query is running:   0%|          |

Downloading:   0%|          |

,type,airport_count
0,large_airport,71


In [ ]:
%%bigquery large_airports_df
SELECT
  ident,
  name,
  latitude_deg,
  longitude_deg
FROM `data_to_ai_workshop.large_airports`;


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
import requests
import pandas as pd

HEADERS = {"User-Agent": "aero-alerts-bootcamp"}
weather_rows = []

for _, row in large_airports_df.iterrows():
    try:
        ident = row.ident
        lat = row.latitude_deg
        lon = row.longitude_deg

        points = requests.get(
            f"https://api.weather.gov/points/{lat},{lon}",
            headers=HEADERS
        ).json()

        stations_url = points["properties"]["observationStations"]
        stations = requests.get(stations_url, headers=HEADERS).json()
        station_id = stations["features"][0]["properties"]["stationIdentifier"]

        obs = requests.get(
            f"https://api.weather.gov/stations/{station_id}/observations/latest",
            headers=HEADERS
        ).json()["properties"]

        weather_rows.append({
            "ident": ident,
            "station": station_id,
            "weather": obs.get("textDescription"),
            "temperature": obs.get("temperature", {}).get("value")
        })

    except Exception as e:
        print(f"Skipping {row.ident}: {e}")

weather_df = pd.DataFrame(weather_rows)
weather_df.head()


,ident,station,weather,temperature
0,PANC,PANC,Light Freezing Rain,0.0
1,KPHX,KPHX,,11.0
2,KSJC,KSJC,Clear,6.0
3,KONT,KONT,Clear,9.0
4,KSAN,KSAN,Mostly Clear,11.0


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `data_to_ai_workshop.airport_weather` (
  ident STRING,
  station STRING,
  weather STRING,
  temperature FLOAT64,
  ingestion_time TIMESTAMP
);


Query is running:   0%|          |

""


In [ ]:
from datetime import datetime

weather_df["ingestion_time"] = datetime.utcnow()
weather_df.head()


/tmp/ipython-input-3085819610.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  weather_df["ingestion_time"] = datetime.utcnow()


,ident,station,weather,temperature,ingestion_time
0,PANC,PANC,Light Freezing Rain,0.0,2026-01-16 15:08:47.091442
1,KPHX,KPHX,,11.0,2026-01-16 15:08:47.091442
2,KSJC,KSJC,Clear,6.0,2026-01-16 15:08:47.091442
3,KONT,KONT,Clear,9.0,2026-01-16 15:08:47.091442
4,KSAN,KSAN,Mostly Clear,11.0,2026-01-16 15:08:47.091442


In [ ]:
client = bigquery.Client(project=project_id)

table_id = "data_to_ai_workshop.airport_weather"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    weather_df, table_id, job_config=job_config
)

job.result()
print(f"Loaded {job.output_rows} rows into {table_id}.")

Loaded 71 rows into data_to_ai_workshop.airport_weather.


In [ ]:
import time
from google.genai.errors import ClientError
from datetime import datetime, timezone

alerts = []

for _, row in weather_df.iterrows():
    prompt = f"""
    Create a concise airport weather alert.

    Airport code: {row.ident}
    Weather: {row.weather}
    Temperature: {row.temperature}°C

    Mention possible delays or safety risks.
    """

    for attempt in range(3):  # max 3 retries
        try:
            response = client.models.generate_content(
                model = "gemini-2.0-flash-lite",
                contents=prompt
            )

            alerts.append({
                "ident": row.ident,
                "alert_text": response.text,
                "alert_time": datetime.now(timezone.utc)
            })
            break  # success → exit retry loop

        except ClientError as e:
            if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                wait_time = 2 ** attempt
                print(f"429 hit. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"Error for {row.ident}: {e}")
                break


In [ ]:
import pandas as pd

alerts_df = pd.DataFrame(alerts)
alerts_df.head()

,ident,alert_text,alert_time
0,PANC,**PANC Airport Weather Alert:**\n\nLight freez...,2026-01-16 15:43:58.520473+00:00
1,KPHX,**KPHX WEATHER ALERT:**\n\nTemperature at KPHX...,2026-01-16 15:43:59.265874+00:00
2,KSJC,"**KSJC Weather Alert:**\n\nClear skies, temper...",2026-01-16 15:44:00.061857+00:00
3,KONT,"**KONT Weather Alert:**\n\nClear skies, 9.0°C....",2026-01-16 15:44:00.666404+00:00
4,KSAN,**KSAN Airport Weather Alert:**\n\nMostly clea...,2026-01-16 15:44:02.030727+00:00


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `data_to_ai_workshop.airport_alerts` (
  ident STRING,
  alert_text STRING,
  alert_time TIMESTAMP
);


Query is running:   0%|          |

""


In [ ]:
client = bigquery.Client(project=project_id)

table_id = "data_to_ai_workshop.airport_alerts"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    alerts_df, table_id, job_config=job_config
)

job.result()
print(f"Loaded {job.output_rows} rows into {table_id}.")

Loaded 71 rows into data_to_ai_workshop.airport_alerts.


In [ ]:
%%bigquery
SELECT *
FROM `data_to_ai_workshop.airport_alerts`
ORDER BY alert_time DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,ident,alert_text,alert_time
0,KMKE,**KMKE Airport Weather Alert:**\n\nLight snow ...,2026-01-16 15:44:49.913663+00:00
1,KSEA,"**KSEA WEATHER ALERT:** Mostly cloudy, 4.0°C. ...",2026-01-16 15:44:49.053417+00:00
2,KRIC,**KRIC Weather Alert:**\n\n**Conditions:** Cle...,2026-01-16 15:44:48.425264+00:00
3,KIAD,**KIAD Weather Alert:**\n\n**Freezing temperat...,2026-01-16 15:44:47.675455+00:00
4,KORF,"**KORF WEATHER ALERT:**\n\nClear skies, -1.0°C...",2026-01-16 15:44:47.047015+00:00
5,KSLC,**KSLC Weather Alert:**\n\nShallow Fog and Fog...,2026-01-16 15:44:46.362963+00:00
6,KSAT,**KSAT WEATHER ALERT:**\n\nTemperature: 12.0°C...,2026-01-16 15:44:45.655145+00:00
7,KDFW,**KDFW Weather Alert:**\n\nClear skies. Temper...,2026-01-16 15:44:44.905709+00:00
8,KAUS,**KAUS Weather Alert:**\n\n**Clear skies. 12.0...,2026-01-16 15:44:44.200698+00:00
9,KIAH,"**KIAH Weather Alert:**\n\nClear skies, 12.0°C...",2026-01-16 15:44:43.501121+00:00


In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `data_to_ai_workshop.v_airport_alerts` AS
SELECT
  a.ident AS airport_code,
  a.name AS airport_name,
  a.latitude_deg,
  a.longitude_deg,
  w.weather,
  w.temperature,
  al.alert_text,
  al.alert_time
FROM `data_to_ai_workshop.large_airports` a
JOIN `data_to_ai_workshop.airport_weather` w USING (ident)
JOIN `data_to_ai_workshop.airport_alerts` al USING (ident);


Query is running:   0%|          |

""
